In [1]:
import sys
sys.path.append("/root/HSE_diploma/")
sys.path.append("/root/HSE_diploma/evaluator/")

In [4]:
from utils.utils import load_model_tokenizer_from_pretrained, draw_metrics_compare_with_glue
from ParamsIterator import ParamsIterator
from EvaluateTestSet import EvaluateTestSet
import pandas as pd
from transformers import GenerationConfig

from tqdm import tqdm_notebook
import json

In [5]:
# default_model = "yahma/llama-7b-hf"
experiment_name = "/root/experiments/t2c_concode_220428_v37/"
default_model = json.load(open("/root/experiments/t2c_concode_220428_v37/experiment_config.json", "r"))['default_model']
params_iteration = {"temperature": [1.0],
                    "max_new_tokens": [300]#None, 20, 30, 35, 40, 50, 60, 70, 80, 90, 100] + [45, 47, 49, 51, 53, 55]
                   }


In [6]:
experiment_name, params_iteration

('/root/experiments/t2c_concode_220428_v37/',
 {'temperature': [1.0], 'max_new_tokens': [300]})

In [7]:
tokenizer, model = load_model_tokenizer_from_pretrained(default_model = default_model, 
                                                        experiment_name = experiment_name
                                                       )

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
def verbose_model_tokenizer(model, tokenizer):
    print("tokenizer.pad_token_id", tokenizer.pad_token_id)
    print("tokenizer.eos_token_id", tokenizer.eos_token_id)
    print("tokenizer.bos_token_id", tokenizer.bos_token_id)
    print("tokenizer.eos_token_id", tokenizer.eos_token_id)

    print("model.config.pad_token_id", model.config.pad_token_id)
    print("model.config.eos_token_id", model.config.eos_token_id)
    print("model.config.bos_token_id", model.config.bos_token_id)
    print("model.config.eos_token_id", model.config.eos_token_id)
    model.print_trainable_parameters()
    
verbose_model_tokenizer(model, tokenizer)

# tokenizer.pad_token_id 0
# tokenizer.eos_token_id 2
# tokenizer.bos_token_id 1
# tokenizer.eos_token_id 2
# model.config.pad_token_id 0
# model.config.eos_token_id 2
# model.config.bos_token_id 1
# model.config.eos_token_id 2
# trainable params: 0 || all params: 6746804224 || trainable%: 0.0


tokenizer.pad_token_id 0
tokenizer.eos_token_id 2
tokenizer.bos_token_id 1
tokenizer.eos_token_id 2
model.config.pad_token_id 0
model.config.eos_token_id 2
model.config.bos_token_id 1
model.config.eos_token_id 2
trainable params: 0 || all params: 6746804224 || trainable%: 0.0


In [9]:
from tqdm import tqdm_notebook

def get_metric_res(model, tokenizer, params_iteration, experiment_name):
    res = []
    for generation_config_dict in tqdm_notebook(ParamsIterator(params_iteration=params_iteration)):
        evaluator = EvaluateTestSet(generation_config = GenerationConfig(**generation_config_dict
                                                                        ),
                                    
                                    #fn_test_data = "temp/t2c_answers.json",
                                    #fn_etalon = "temp/answers.json"
                                   )

        metric_res = evaluator.evaluate(model=model, 
                                        tokenizer=tokenizer                                        
                                       )
        for key, val in generation_config_dict.items():
            assert key not in metric_res
            metric_res[key] = val

        metric_res['experiment_name'] = experiment_name
        print(generation_config_dict, metric_res)

        res.append(metric_res)
    
    return res

In [10]:
tokenizer.pad_token_id = model.config.pad_token_id = 0

In [11]:
model.eval();

In [12]:
res3 = get_metric_res(model=model,
                     tokenizer=tokenizer,
                     params_iteration=params_iteration,
                     experiment_name=experiment_name
                    )
res3

/tmp/ipykernel_3481/3839643825.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for generation_config_dict in tqdm_notebook(ParamsIterator(params_iteration=params_iteration)):


  0%|          | 0/1 [00:00<?, ?it/s]


  0%|                                                    | 0/10 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.

100%|███████████████████████████████████████████| 10/10 [03:30<00:00, 21.08s/it]

100%|██████████████████████████████████████| 100/100 [00:00<00:00, 32221.74it/s]

{'max_new_tokens': 300, 'temperature': 1.0} {'EM': 0.0, 'BLEU': 0.0006187833664486746, 'brevity_penalty': 0.30601594809335797, 'ratio': 0.45785070785070786, 'translation_length': 1423, 'reference_length': 3108, 'precisions_0': 0.03651685393258427, 'precisions_1': 0.0007552870090634441, 'precisions_2': 0.0007710100231303007, 'precisions_3': 0.0007861635220125787, 'max_new_tokens': 300, 'temperature': 1.0, 'experiment_name': '/root/experiments/t2c_concode_220428_v37/'}


[{'EM': 0.0,
  'BLEU': 0.0006187833664486746,
  'brevity_penalty': 0.30601594809335797,
  'ratio': 0.45785070785070786,
  'translation_length': 1423,
  'reference_length': 3108,
  'precisions_0': 0.03651685393258427,
  'precisions_1': 0.0007552870090634441,
  'precisions_2': 0.0007710100231303007,
  'precisions_3': 0.0007861635220125787,
  'max_new_tokens': 300,
  'temperature': 1.0,
  'experiment_name': '/root/experiments/t2c_concode_220428_v37/'}]

In [15]:
from prompter import prompter, get_response

In [16]:
s = """### Response:
TernaryBool function ( ) { return canBeInstantiated ( ) ; }</s><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk>
===
<unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generate java code
interpret a short as its binary form 

### Input:
 PlaceHolder placeHolder 

 long asLong 
 int toInt 
 String toBinaryString 
 String toBinaryString 
 String toBinaryString 
 byte[] fromInt 
 byte[] fromLong

### Response:
short function ( final short arg0 ) { return ( short ) ( arg0 & 0xFFFF ) ; }</s><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk>"""

In [19]:
o = EvaluateTestSet()
o.preprocess(s)

### Response:
TernaryBool function ( ) { return canBeInstantiated ( ) ; }</s><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk>
===
<unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><un

'### Response: TernaryBool function ( ) { return canBeInstantiated ( ) ; }</s> === '

In [17]:
get_response(s)

### Response:
TernaryBool function ( ) { return canBeInstantiated ( ) ; }</s><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk>
===
<unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><un

'short function ( final short arg0 ) { return ( short ) ( arg0 & 0xFFFF ) ; }</s><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk><unk>'

/tmp/ipykernel_1969/3839643825.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for generation_config_dict in tqdm_notebook(ParamsIterator(params_iteration=params_iteration)):


  0%|          | 0/1 [00:00<?, ?it/s]


  0%|                                                    | 0/10 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.

100%|███████████████████████████████████████████| 10/10 [02:15<00:00, 13.54s/it]

100%|██████████████████████████████████████| 100/100 [00:00<00:00, 34314.85it/s]

{'max_new_tokens': 200, 'temperature': 1.0} {'EM': 0.0, 'BLEU': 0.0006187833664486746, 'brevity_penalty': 0.30601594809335797, 'ratio': 0.45785070785070786, 'translation_length': 1423, 'reference_length': 3108, 'precisions_0': 0.03651685393258427, 'precisions_1': 0.0007552870090634441, 'precisions_2': 0.0007710100231303007, 'precisions_3': 0.0007861635220125787, 'max_new_tokens': 200, 'temperature': 1.0, 'experiment_name': '/root/experiments/t2c_concode_220428_v34/'}


[{'EM': 0.0,
  'BLEU': 0.0006187833664486746,
  'brevity_penalty': 0.30601594809335797,
  'ratio': 0.45785070785070786,
  'translation_length': 1423,
  'reference_length': 3108,
  'precisions_0': 0.03651685393258427,
  'precisions_1': 0.0007552870090634441,
  'precisions_2': 0.0007710100231303007,
  'precisions_3': 0.0007861635220125787,
  'max_new_tokens': 200,
  'temperature': 1.0,
  'experiment_name': '/root/experiments/t2c_concode_220428_v34/'}]